# Assignment 2 - KNN
### Kieran Gregg

In [1]:
import pandas as pd

# read the titanic data
df = pd.read_csv("titanic.csv")
df

FileNotFoundError: [Errno 2] No such file or directory: 'titanic.csv'

In [ ]:
df.describe()

In [ ]:
df.info()

#### ^ Age is missing a fair amount of values, but it's still important to keep around, so I'll fill these with averages

In [ ]:
# getting rid of these columns, as we don't need them or they carry so little weight
df.drop(columns=['Name', "Ticket", "Cabin", "Embarked", "PassengerId"], inplace=True)
df

In [ ]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()

# Uses the sklearn LabelBinarizer to take the Sex entry and convert to 1's and 0's instead of using the male/female String name
transformed = lb.fit_transform(df["Sex"])
df_transformed = pd.DataFrame(transformed, columns=["BinarySex"])
df = pd.concat([df, df_transformed], axis=1).drop("Sex", axis=1)
df

In [ ]:
male = df[df["BinarySex"]==1]
female = df[df["BinarySex"]==0]
# getting the average male and female age, to fill in the few blank values
avg_male_age = male.Age.mean()
avg_female_age = female.Age.mean()
print(avg_male_age)
print(avg_female_age)

In [ ]:
# filling the blank male values with the average male age
male_df = df.loc[df["BinarySex"]==1].fillna(value=avg_male_age)
# filling the blank female values with the average female age
female_df = df.loc[df["BinarySex"]==0].fillna(avg_female_age)

In [ ]:
# combining the male and female dataframes back into one
df = pd.concat([male_df, female_df])
df

In [ ]:
# our target is whether or not the passenger survived
targets = df["Survived"]
# our features are everything else... ticket class, age, siblings, parents, fare, sex..
features = df.drop(columns=["Survived"])

In [ ]:
# random 33% of the data split for training
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(features, targets, train_size=0.33, random_state=42, shuffle=True)
type(train_x)

In [ ]:
# outputting the training sets and the feature data to csv's so my java code can read them in
train_x.to_csv("train_x.csv", header=None, index=False)
train_y.to_csv("train_y.csv", header=None, index=False)
features.to_csv("features.csv", header=None, index=False)
targets.to_csv("targets.csv", header=None, index=False)

# Testing & Comparing

### sklearn's KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# runnign the built in sklearn KNN algorithm
acc = dict()
for k in range(2, 20):
    clf = KNeighborsClassifier(n_neighbors=k)
    clf.fit(train_x, train_y)
    acc[k] = accuracy_score(targets, clf.predict(features))
acc

##### ^ The scores for the sklearn KNN above, roughly 0.69-0.72

### My Amazing Java KNN

In [ ]:
import os

train_x_csv_path = "train_x.csv"
train_y_csv_path = "train_y.csv"
features_csv_path = "features.csv"
targets_csv_path = "targets.csv"
random_state = 42
prediction_csv_path = "prediction.csv"

cwd = os.getcwd()
os.chdir(cwd)
os.system('javac KNN.java')   


kieran_knn_acc = dict()
for k in range(2, 20):
    run_command = "java " + "KNN " + train_x_csv_path + " " + train_y_csv_path + " " + features_csv_path + " " + str(k) + " " + str(random_state) + " " + prediction_csv_path
    os.system(run_command)
    
    predicted = pd.read_csv("prediction.csv", header=None)
    kieran_knn_acc[k] = accuracy_score(targets, predicted)
kieran_knn_acc

##### ^ The scores for my Java KNN above, roughly 0.685-0.72, VERY close to the sklearn scores